In [1]:
#!pip install --upgrade git+https://github.com/EmGarr/kerod.git

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
import  functools
import  tensorflow as tf
import  tensorflow_datasets as tfds
from    tensorflow.keras.utils import to_categorical
import  matplotlib.pyplot as plt
from    tensorflow.keras import layers
from    tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import pandas as pd
from    tqdm import tqdm
import  time
from    sklearn.metrics import accuracy_score

from common.config import Config
from common.logger import Logger
from common.communication import Client
from common.communication import Server
from common.helper import ImagesInfo 
from common.timekeeper import TimeKeeper
from common.helper import read_image, filt_text, get_predictions
from CaptionModel import CaptionModel
from common.helper import read_image, filt_text, get_predictions,process_predictions


In [4]:
data_dir='/home/suphale/coco'
N_LABELS = 80
N_EPOCHS = 1
TRAIN_MODE = False
split_train = "train[:1%]"
split_val = "validation[:1%]"
h_image_height = 299
h_image_width = 299

SPLIT_LAYER = 4


In [5]:
tk = TimeKeeper()
cfg = Config()
client = Client(cfg)
imagesInfo = ImagesInfo(cfg)

In [6]:
from tensorflow import keras  # or import keras for standalone version
from tensorflow.keras.layers import Input

org_model = tf.keras.models.load_model(cfg.temp_path + '/extractor_model')
# org_model = tf.keras.models.load_model(cfg.saved_model_path + '/model')

In [7]:
model_config = org_model.get_config()

In [8]:
max_layer_index = len(model_config['layers']) - 1

In [9]:

head_model_config = {}
head_model_config['name'] = 'head_model'
head_model_config['layers'] = []
head_model_config['input_layers'] = [[model_config['layers'][0]['name'],0,0]]
head_model_config['output_layers'] = [[model_config['layers'][SPLIT_LAYER-1]['name'],0,0]]

for index in range(SPLIT_LAYER):
    print("%d %s" % (index, model_config['layers'][index]['name']) )
    head_model_config['layers'].append(model_config['layers'][index])

0 input_1
1 conv2d
2 batch_normalization
3 activation


In [14]:
import copy

tail_model_config = copy.deepcopy(model_config)
tail_model_config['name'] = 'tail_model'
tail_model_config['input_layers'] = [[model_config['layers'][SPLIT_LAYER]['name'],0,0]]
# tail_model_config['output_layers'] = [[model_config['layers'][max_layer_index]['name'],0,0]]
tail_model_config['output_layers'] = model_config['output_layers']

In [15]:
print(tail_model_config['output_layers'])

[['mixed10', 0, 0], ['dense_1', 0, 0]]


In [16]:
new_input_layer = {
                      'name': 'new_input',
                      'class_name': 'InputLayer',
                      'config': {
                          'batch_input_shape': tuple(org_model.layers[SPLIT_LAYER-1].output.shape),
                          'dtype': 'float32',
                          'sparse': False,
                          'name': 'new_input'
                      },
                      'inbound_nodes': []
                  }
tail_model_config['layers'][0] = new_input_layer

In [17]:
for index in range(1,SPLIT_LAYER):
    print("%d %s" % (index, tail_model_config['layers'][1]['name']) )
    tail_model_config['layers'].pop(1)

1 conv2d
2 batch_normalization
3 activation


In [18]:
tail_model_config['layers'][1]['inbound_nodes'] = [[['new_input', 0, 0, {}]]]
tail_model_config['input_layers'] = [['new_input', 0, 0]]

In [19]:
import pprint
with open(cfg.temp_path + '/model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(model_config))
    print(model_config,file=fh)
with open(cfg.temp_path + '/head_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    print(head_model_config,file=fh)
with open(cfg.temp_path + '/tail_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    print(tail_model_config,file=fh)


In [20]:
head_model = org_model.__class__.from_config(head_model_config, custom_objects={})

In [21]:
tail_model = org_model.__class__.from_config(tail_model_config, custom_objects={})

In [22]:
with open(cfg.temp_path + '/org_model.txt','w') as fh:
    org_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open(cfg.temp_path + '/head_model.txt','w') as fh:
    head_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open(cfg.temp_path + '/tail_model.txt','w') as fh:
    tail_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)


In [23]:
for index, layer in enumerate(org_model.layers[:SPLIT_LAYER]):
    # print("[%d] %s %s" % (index, layer.name, str(np.shape(weight))))
    weight = layer.get_weights()
    new_head_model_layer = head_model.layers[index]
    new_head_model_layer.set_weights(weight)

In [24]:
import numpy as np
for index, layer in enumerate(org_model.layers[SPLIT_LAYER:max_layer_index+1]):
    weight = layer.get_weights()
    # print("org_model [%d] %s %s" % (index, layer.name, str(tf.shape(weight))))
    tail_model_layer = tail_model.layers[index+1]
    tail_model_layer_weight = tail_model_layer.get_weights()
    # print("tail_model [%d] %s %s" % (index, tail_model_layer.name, str(tf.shape(tail_model_layer_weight))))
    tail_model_layer.set_weights(weight)

In [25]:
head_model.save(cfg.temp_path + '/head_model')
tail_model.save(cfg.temp_path + '/tail_model')

INFO:tensorflow:Assets written to: /home/suphale/WorkSpace/temp/head_model/assets
INFO:tensorflow:Assets written to: /home/suphale/WorkSpace/temp/tail_model/assets


In [26]:
class BoxField:
    BOXES = 'bbox'
    KEYPOINTS = 'keypoints'
    LABELS = 'label'
    MASKS = 'masks'
    NUM_BOXES = 'num_boxes'
    SCORES = 'scores'
    WEIGHTS = 'weights'

class DatasetField:
    IMAGES = 'images'
    IMAGES_INFO = 'images_information'
    IMAGES_PMASK = 'images_padding_mask'

def my_preprocess(inputs):
    image = inputs['image']
    image = tf.image.resize(image, (h_image_height, h_image_width))
    image = tf.cast(image, tf.float32)
    image /= 127.5
    image -= 1.

    targets = inputs['objects']
    img_path = inputs['image/filename']

    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32)

    inputs = {DatasetField.IMAGES: image, DatasetField.IMAGES_INFO: image_information}

    # ground_truths = {
    #     BoxField.BOXES: targets[BoxField.BOXES] * tf.tile(image_information[tf.newaxis], [1, 2]),
    #     BoxField.LABELS: tf.cast(targets[BoxField.LABELS], tf.int32),
    #     BoxField.NUM_BOXES: tf.shape(targets[BoxField.LABELS]),
    #     BoxField.WEIGHTS: tf.fill(tf.shape(targets[BoxField.LABELS]), 1.0)
    # }
    ground_truths = tf.cast(targets[BoxField.LABELS], tf.int32)
    # ground_truths = tf.one_hot(ground_truths, depth=N_LABELS, dtype=tf.int32)
    # ground_truths = tf.reduce_sum(ground_truths, 0)
    # ground_truths = tf.greater( ground_truths, tf.constant( 0 ) )    
    # ground_truths = tf.where (ground_truths, 1, 0) 
    return image, ground_truths, img_path

def expand_dims_for_single_batch(image, ground_truths, img_path):
    image = tf.expand_dims(image, axis=0)
    ground_truths = tf.expand_dims(ground_truths, axis=0)
    return image, ground_truths, img_path

In [27]:
ds_val, ds_info = tfds.load(name="coco/2017", split=split_val, data_dir=data_dir, shuffle_files=False, download=False, with_info=True)
ds_val = ds_val.map(functools.partial(my_preprocess), num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [28]:
from   nltk.translate.bleu_score import sentence_bleu
def process_caption_predictions(caption_tensor, img_path):
    pred_caption=' '.join(caption_tensor).rsplit(' ', 1)[0]
    real_appn = []
    real_caption_list = imagesInfo.annotations_dict[img_path]
    for real_caption in real_caption_list:
        real_caption=filt_text(real_caption)
        real_appn.append(real_caption.split())
    reference = real_appn
    candidate = pred_caption.split()
    score = sentence_bleu(reference, candidate, weights=[1]) #set your weights)
    return score,real_caption,pred_caption

In [31]:
Test = True
captionModel = CaptionModel()
if (Test == True):
    ds_val = ds_val.take(1)
    for sample_img_batch, ground_truth, img_path in tqdm(ds_val):
        # count += 1

        tensor_shape = len(ground_truth.get_shape().as_list())
        if(tensor_shape > 1):
            ground_truth = tf.squeeze(ground_truth,[0])
        ground_truth = list(set(ground_truth.numpy()))

        img_path = img_path.numpy().decode()
        print(img_path)
        h = head_model(sample_img_batch)
        features, result = tail_model(h)
        # features, result = model(sample_img_batch)
        predictions, predictions_prob = get_predictions(cfg, result)
        accuracy, top_1_accuracy,top_5_accuracy,precision,recall, top_predictions, predictions_str = process_predictions(cfg, imagesInfo, ground_truth,predictions, predictions_prob)
        print(predictions_str)

        features = tf.reshape(features, [sample_img_batch.shape[0],8*8, 2048])
        caption_tensor = captionModel.evaluate(features)

        print(type(caption_tensor))
        score,real_caption,pred_caption = process_caption_predictions(caption_tensor, img_path)

        print("BLEU: %.2f" % (score))
        print ('Real:', real_caption)
        print ('Pred:', pred_caption)    

  0%|          | 0/1 [00:00<?, ?it/s]

000000531036.jpg
person(69.00) car(58.00) bus(89.00) truck(48.00) 


100%|██████████| 1/1 [00:01<00:00,  1.14s/it]

<class 'list'>
BLEU: 0.60
Real: a double decker bus is stopped along a curb
Pred: a red double decker bus is driving down the street
